### Install

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl
!pip install -q loralib einops
!pip install -q pandas
!pip install gspread
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━

###Env

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from huggingface_hub import login
login("xxx")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wandb login
#e4460d24dea96f9c121f12f8aeb7e4176654b332

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Model

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, AutoConfig
from trl import SFTTrainer
import os

In [ ]:
model_id="vinai/PhoGPT-4B-Chat"
output_model="PhoGPT-7B5-LoRA-v1"

In [ ]:
def get_model_and_tokenizer(mode_id):
    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token

    config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
    config.init_device = "cuda"

    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype="float16"
    # )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config = config,
        # load_in_4bit=True,
        torch_dtype=torch.float16,
        # quantization_config=bnb_config,
        trust_remote_code=True
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


hf_prefixlm_converter.py:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- adapt_tokenizer.py
- meta_init_context.py
- custom_embedding.py
- param_init_fns.py
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

### Data

####Preprocess

In [ ]:
import pandas as pd
import json
import re

# Đường dẫn đến tệp Excel
excel_file_path = "/content/drive/MyDrive/FAQ-final.xlsx"

# Đọc tệp Excel vào DataFrame
df = pd.read_excel(excel_file_path)

# Tạo danh sách chứa các cặp câu hỏi và câu trả lời
qa_pairs = []

# Duyệt qua mỗi hàng trong DataFrame
for index, row in df.iterrows():
  # Lấy nội dung của cột câu hỏi và câu trả lời
  questions = row['QUESTION'].split("\n")  # Tách các câu hỏi trong cell
  response = row['ANSWER']
  questions = [re.sub(r'^\d+\.\s*', '', question) for question in questions]
  questions = [question.rstrip('"') for question in questions]

  # Tạo các cặp câu hỏi và câu trả lời
  for question in questions:
    # Kiểm tra nếu câu hỏi không rỗng thì thêm vào danh sách
    if question.strip():
      qa_pairs.append({"question": question.strip(), "response": response.strip()})

# Ghi danh sách các cặp câu hỏi và câu trả lời vào tệp JSON
output_json_file_path = "/content/drive/MyDrive/FAQ_processed.json"
with open(output_json_file_path, 'w', encoding='utf-8') as json_file:
  json.dump(qa_pairs, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được xử lý và ghi vào tệp JSON mới:", output_json_file_path)


Dữ liệu đã được xử lý và ghi vào tệp JSON mới: /content/drive/MyDrive/FAQ_processed.json


In [ ]:
# data_set.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['QUESTION'], df['ANSWER'], test_size=0.2, random_state=42)

# Vector hóa văn bản
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Huấn luyện mô hình
model_eval = MultinomialNB()
model_eval.fit(X_train_vec, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model_eval.predict(X_test_vec)

# Độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print("Độ chính xác trên tập kiểm tra:", accuracy*100)

# Precision
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision trên tập kiểm tra:", precision*100)

# Recall
recall = recall_score(y_test, y_pred, average='weighted')
print("Recall trên tập kiểm tra:", recall*100)

# F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score trên tập kiểm tra:", f1*100)


Độ chính xác trên tập kiểm tra: 86.36363636363636
Precision trên tập kiểm tra: 88.69318181818181
Recall trên tập kiểm tra: 86.36363636363636
F1-score trên tập kiểm tra: 86.09265734265733


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


####Read data

In [ ]:
import pandas as pd
from datasets import Dataset

json_file_path = output_json_file_path
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# In ra dữ liệu đọc được
print(data)

[{'question': 'Khách sạn có bữa sáng phục vụ tại phòng không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Làm thế nào để đặt dịch vụ bữa sáng phục vụ tại phòng?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Có phải khách sạn cung cấp bữa sáng tại phòng cho khách không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Quy trình để yêu cầu bữa sáng tận phòng là gì?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Bữa sáng có chuẩn bị tận phòng không?', 'response': 'Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.'}, {'question': 'Làm thế nào để biết thông tin chi tiết về dịch vụ bữa sáng tận phòng?', 'response': 'Mandala không phục vụ bữa sáng tại 

In [ ]:
my_dict = []
question_responses = data
for qr in question_responses:
    my_dict.append({'text': f"Câu hỏi: {qr['question']}\nTrả lời:{qr['response']}"})

data_set = pd.DataFrame(my_dict)
data = Dataset.from_pandas(data_set)

In [ ]:
print(data['text'])

['Câu hỏi: Khách sạn có bữa sáng phục vụ tại phòng không?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏi: Làm thế nào để đặt dịch vụ bữa sáng phục vụ tại phòng?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏi: Có phải khách sạn cung cấp bữa sáng tại phòng cho khách không?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏi: Quy trình để yêu cầu bữa sáng tận phòng là gì?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏi: Bữa sáng có chuẩn bị tận phòng không?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏi: Làm thế nào để biết thông tin chi tiết về dịch vụ bữa sáng tận phòng?\nTrả lời:Mandala không phục vụ bữa sáng tại phòng thường, chỉ áp dụng với hạng phòng Master Suites.', 'Câu hỏ

### Training

### Train

In [ ]:
peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        target_modules=[
          "attn.Wqkv",
          "attn.out_proj",
          "ffn.up_proj",
          "ffn.down_proj",
          ],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    report_to = 'wandb',
    save_steps=50,
    logging_steps=10,
    num_train_epochs=1000,
    max_steps=1000,
    fp16=False,
)

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/440 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: Currently logged in as: khaiquang-dev. Use `wandb login --relogin` to force relogin


/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Step,Training Loss
10,2.394600
20,2.226900
30,2.172600
40,1.835600
50,1.605600
60,1.482000
70,1.329200
80,1.162100
90,1.253200
100,1.150900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
/root/.cache/hug

TrainOutput(global_step=1000, training_loss=0.5086127319335938, metrics={'train_runtime': 269.0735, 'train_samples_per_second': 3.716, 'train_steps_per_second': 3.716, 'total_flos': 1282666684723200.0, 'train_loss': 0.5086127319335938, 'epoch': 2.2727272727272725})

###Merging the LoRA with the base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
from peft import LoraConfig, get_peft_model
# pretrained_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=True,
#                             device_map="cuda:0", trust_remote_code=True, use_auth_token=None,)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
model_path = "PhoGPT-7B5-LoRA-v1/checkpoint-1000"

peft_model = PeftModel.from_pretrained(model, model_path)

In [ ]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MPTForCausalLM(
      (transformer): MPTModel(
        (wte): SharedEmbedding(20480, 3072)
        (emb_drop): Dropout(p=0.0, inplace=False)
        (blocks): ModuleList(
          (0-31): 32 x MPTBlock(
            (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (Wqkv): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=9216, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=9216, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Paramet

In [ ]:
peft_model.print_trainable_parameters()

trainable params: 0 || all params: 3,738,408,960 || trainable%: 0.0


###LLM Response

In [ ]:
def generate_response(user_input):
  messages = [
    {"role": "system", "content": "Bạn là một nhân viên lễ tân phục vụ khách sạn tận tình"},
    {"role": "user", "content": user_input}
  ]
  input_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  input_ids = tokenizer(input_prompt, return_tensors="pt")
  outputs = peft_model.generate(
    inputs=input_ids["input_ids"].to("cuda"),
    attention_mask=input_ids["attention_mask"].to("cuda"),
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.9,
    max_new_tokens=1024,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
  )
  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
  response = response.split("### Trả lời:")[1]
  return response


In [ ]:
!pip install python-telegram-bot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.7/618.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

In [ ]:
# user_input = 'Khách sạn có bữa sáng phục vụ tại phòng không?'
user_input = "Khách sạn có Room service không?"
# user_input = 'Nếu tôi hủy đặt phòng, liệu có mất phí không?'
Markdown(generate_response(user_input))

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Mandala có dịch vụ Room Service 24/7. Bạn có thể scan QR code Menu tại phòng để sử dụng dịch vụ. Sảnh Lễ tân luôn sẵn sàng hỗ trợ Bạn.

In [ ]:
import logging
import os
from collections import defaultdict

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler


In [ ]:
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

# This dictionaty stores the messages. The key is the chat_id and the value is a list of messages.
chat_history = defaultdict(list)


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Đây là dịch vụ trả lời tự động của hệ thống khách sạn Mandala. Tôi có thể giúp gì cho bạn?")


async def update_and_get_chat_history(message: str, chat_id: int, top_k_messages: int = 5) -> list:
    chat_history[chat_id].append(message)
    return chat_history[chat_id][-top_k_messages:-1]


async def ai_chat(update: Update, context: ContextTypes.DEFAULT_TYPE, with_context: bool = True):
    chat_id = update.effective_chat.id
    message = update.effective_message.text

    history = await update_and_get_chat_history(message, chat_id)

    if with_context:
        str_history = "\n".join(history)
        formatted_prompt = f"Recent user messages:\n\n{str_history}\n\nUser message to answer:\n\n{message}"
        system_prompt = "You are a helpful assistant. Keep the conversation with user attending to the user message. Use recent messages as context to provide better answers and adequate tone."
    else:
        formatted_prompt = message
        system_prompt = "You are a helpful assistant."

    response = generate_response(message)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=response)

In [ ]:
application = ApplicationBuilder().token("5926044398:AAGoDAGjYoWDZQcW3x3WPaTEGEtxzoE7ybw").build()

start_handler = CommandHandler("start", start)
application.add_handler(start_handler)

message_handler = MessageHandler(None, ai_chat)
application.add_handler(message_handler)

async with application:
    await application.initialize() # inits bot, update, persistence
    await application.start()
    await application.updater.start_polling()

RuntimeError: This Application is still running!